In [1]:
from scipy.stats import entropy

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
df = []  # Initialize an empty list; presumably, this should be a DataFrame instead.

In [3]:
# Function to calculate density for a given data column over a developed area column.
def calculate_density(df, data_column, developed_area_column):
    # Calculate density and store it in a new column 'density'
    df['density'] = df[data_column] / df[developed_area_column]
    return df

In [4]:
# Function to retrieve indices of neighboring cells given a cell's index.
def get_neighboring_cells(df, index):
    # Extracts the current cell's row and column position
    current_cell = df.loc[index]
    current_row = current_cell['row']
    current_column = current_cell['column']

    # Defines relative positions of the 8 neighboring cells
    neighbor_positions = [(-1, -1), (-1, 0), (-1, 1),
                          (0, -1), (0, 1),
                          (1, -1), (1, 0), (1, 1)]

    neighbors = []
    # Iterates through neighbor positions to find valid neighbors
    for row_offset, col_offset in neighbor_positions:
        neighbor_row = current_row + row_offset
        neighbor_col = current_column + col_offset

        # Checks if neighbor is within grid bounds
        if (0 <= neighbor_row < df['row'].max()) and (0 <= neighbor_col < df['column'].max()):
            # Finds index of the neighbor cell if it exists
            neighbor_index = df[(df['row'] == neighbor_row) & (df['column'] == neighbor_col)].index
            if not neighbor_index.empty:
                neighbors.append(neighbor_index[0])

    return neighbors

In [5]:
# Function to calculate dissimilarity index based on land use categories.
def calculate_dissimilarity_index(df, land_use_columns):
    developed_cells = df[land_use_columns]
    K = len(developed_cells)  # Total number of developed cells
    dissimilarity_sum = 0
    # Iterates through each developed cell to calculate dissimilarity
    for index, row in developed_cells.iterrows():
        neighboring_indices = get_neighboring_cells(df, index)
        # Dissimilarity calculation based on land use category differences
        dissimilarity = sum([1/8 if df.loc[n_idx, 'land_use_category'] != row['land_use_category'] else 0
                             for n_idx in neighboring_indices])
        dissimilarity_sum += dissimilarity

    # Calculates overall dissimilarity index
    dissimilarity_index = dissimilarity_sum / K
    return dissimilarity_index

In [6]:
# Function to calculate entropy for land use distribution.
def calculate_entropy(df, land_use_columns):
    # Calculates probabilities for each land use type and then entropy
    probabilities = df[land_use_columns].div(df[land_use_columns].sum(axis=1), axis=0)
    df['entropy'] = probabilities.apply(entropy, axis=1)
    return df

In [7]:
# Function to calculate the proportion of parcels with vertical land use mixture.
def calculate_vertical_mixture(df, land_use_intensity_columns):
    # Counts unique land use categories per parcel and identifies mixed-use parcels
    land_use_per_parcel = df.groupby('parcel_id')['land_use_category'].nunique()
    mixed_use_parcels = land_use_per_parcel[land_use_per_parcel > 1]
    df['vertical_mixture_proportion'] = len(mixed_use_parcels) / len(land_use_per_parcel)
    return df

In [8]:
# Function to calculate commercial intensity as a proportion of the developed area.
def calculate_intensity(df, commercial_land_use_columns, developed_area_column):
    df['commercial_intensity'] = df[commercial_land_use_columns].sum(axis=1) / df[developed_area_column]
    return df

In [9]:
# Function to calculate proximity to commercial-retail uses as a proportion of the developed area.
def calculate_proximity(df, developed_area_column, proximity_columns):
    df['proximity_to_commercial'] = df[proximity_columns].sum(axis=1) / df[developed_area_column]
    return df

In [10]:
# Function to calculate design proportions for various street patterns.
def calculate_design_proportion(df, street_pattern_columns):
    df['design_proportion'] = df[street_pattern_columns].apply(lambda x: x / x.sum(), axis=1)
    return df

In [11]:
# Function to calculate design acre rates based on developed acres.
def calculate_design_acrerates(df, street_pattern_columns, developed_acres_column):
    df['design_acrerates'] = df[street_pattern_columns].sum(axis=1) / df[developed_acres_column]
    return df

In [12]:
# Function to calculate the mean of specified limits.
def calculate_design_mean(df, limits_columns):
    df['average'] = df